<a href="https://colab.research.google.com/github/lcylsy0725/lcylsy0725/blob/main/Data_Collection_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sec-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import json

In [51]:
import csv 
import re

In [52]:
api_key = "8fc9a2565c1278fcfd310fd773d7f24e4e66ebd1a00a39c9355c71e052cbba08"

In [53]:
queryApi = QueryApi(api_key=api_key)

In [69]:
extractorApi = ExtractorApi(api_key)

In [50]:
ticker_100 = {'AAPL','MSFT','AMZN', 'NVDA','GOOGL','TSLA','BRK.B','GOOG','XOM','UNH','JNJ','JPM',"META","V","PG","HD","MA","CVX", "MRX", "LLY",
          "ABBV","BAC","PFE","AVGO","PEP","KO","TMO","COST","WMT","DIS","CSCO","MCD","ABT","WFC","ACN","ADBE","CRM","VZ","CMCSA","DHR","LIN","TXN","NFLX",
          "NKE","PM","BMY","NEE","RTX","QCOM","COP","AMD","ORCL","T","UPS","HON","LOW","MS","AMGN","CAT","GS","UNP","SBUX", "IBM", "SCHW","BA", "SPGI", "INTC", "INTU", "PLD", "ELV","CVS", "DE",
          "LMT","MDT","BLK","AXP","GILD","AMAT","AMT","C","BKNG","ADI","TJX","ADP","NOW","CI","GE", "MDLZ","SYK", "CB", "TMUS", " PYPL", "ISRG", "MO", "MMC", "PGR", "SLB", "REGN", "ZTS","TGT"}

ticker_3 = {"AAPL"}

In [54]:
base_query = {
  "query": { 
      "query_string": { 
          "query": "PLACEHOLDER", # this will be set during runtime 
          "time_zone": "America/New_York"
      } 
  },
  "from": "0",
  "size": "200", # dont change this
  # sort by filedAt
  "sort": [{ "filedAt": { "order": "desc" } }]
}

In [55]:
# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "a")

# start with filings filed in 2021, then 2020, 2019, ... up to 2010 
# uncomment line below to fetch all filings filed in 2022-2010
# for year in range(2021, 2009, -1):
for tckr in ticker_3:
  print("starting {tckr}".format(tckr=tckr))
  
  # a single search universe is represented as a month of the given year
  for month in range(1, 13, 1):
    # get 10-Q and 10-Q/A filings filed in year and month
    # resulting query example: "formType:\"10-Q\" AND filedAt:[2021-01-01 TO 2021-01-31]"
    universe_query = \
        "formType:\"10-K\" AND " + \
        "ticker:{tckr}" \
        .format(tckr = tckr)
  
    print(universe_query)
    # set new query universe for year-month combination
    base_query["query"]["query_string"]["query"] = universe_query;

    # paginate through results by increasing "from" parameter 
    # until we don't find any matches anymore
    # uncomment line below to fetch 10,000 filings
    # for from_batch in range(0, 9800, 200): 
    for from_batch in range(0, 400, 200):
      # set new "from" starting position of search 
      base_query["from"] = from_batch;

      response = queryApi.get_filings(base_query)

      # no more filings in search universe
      if len(response["filings"]) == 0:
        break;

      # for each filing, only save the URL pointing to the filing itself 
      # and ignore all other data. 
      # the URL is set in the dict key "linkToFilingDetails"
      urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))

      # transform list of URLs into one string by joining all list elements
      # and add a new-line character between each element.
      urls_string = "\n".join(urls_list) + "\n"
      
      log_file.write(urls_string)

 

starting AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL
formType:"10-K" AND ticker:AAPL


In [77]:
# pattern = r"\\[a-z]\\[a-z](&#[0-9])*"
pattern = r"(&#[0-9]+)*"

def edit_func(value):
  value = value 
  editted = re.sub(pattern, "", value, count=0, flags=0) 
  return editted


In [76]:
condensed_url = urls_list[1:3]
extracted_data = []
for url in condensed_url:
    company_name = tckr
    form_of_filing = "10K"
    url = url
    risk_section_not_edit = extractorApi.get_section(url, "1A", "text") 
    risk_section = edit_func(risk_section_not_edit) 
    mda_section_not_edit = extractorApi.get_section(url, "7", "text")  
    mda_section = edit_func(mda_section_not_edit)
    extracted_data.append([company_name, form_of_filing, url, risk_section, mda_section])

# Write the extracted data to a CSV file
with open("filing_data.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Company Ticker", "Form of Filing", "URL", "Risk Factor Text", "MD&A Text"])
    for data_row in extracted_data:
        writer.writerow(data_row) 

In [66]:
# urls_list = list(map(lambda x: x["linkToFilingDetails"], query_result["filings"])) 


# urls_list

In [65]:
# query_result

In [30]:
# from sec_api import ExtractorApi
# # Extractor API documentation: https://sec-api.io/docs/sec-filings-item-extraction-api 

# extractorApi = ExtractorApi("065ad8e844002b8ad717d0c0c52b9218cb26a893dd7c028826f2577ed012a4a8")

# filing_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019319000119/a10-k20199282019.htm"

# # get the standardized HTML of section 1A "Risk Factors"
# section_html = extractorApi.get_section(filing_url, "1A", "html")

# # get the standardized and cleaned text of section 1A "Risk Factors"
# section_text = extractorApi.get_section(filing_url, "1A", "text")